# Co když webová stránka používá Javascript?

## Sankční seznamy EU

Př.: Lidé z Afghánistánu, kterým se mají zmrazit finance

https://sanctionsmap.eu/#/main/details/1/lists?search=%7B%22value%22:%22%22,%22searchType%22:%7B%7D%7D


![sanction lists devtools](sanctions1.png)

In [60]:
from pprint import pprint
import re

import pandas as pd
import requests

In [13]:
resp = requests.get('https://sanctionsmap.eu/#/main/details/1/lists?search=%7B%22value%22:%22%22,%22searchType%22:%7B%7D%7D1')

In [14]:
print(resp.text)

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
  <meta name="description" content="">
  <meta name="author" content="">
  <meta name="version" content="1.1.0">

  <link rel="apple-touch-icon" sizes="57x57" href="assets/images/icons/apple-touch-icon-57x57.png?v=LbGYE7o8gO">
  <link rel="apple-touch-icon" sizes="60x60" href="assets/images/icons/apple-touch-icon-60x60.png?v=LbGYE7o8gO">
  <link rel="apple-touch-icon" sizes="72x72" href="assets/images/icons/apple-touch-icon-72x72.png?v=LbGYE7o8gO">
  <link rel="apple-touch-icon" sizes="76x76" href="assets/images/icons/apple-touch-icon-76x76.png?v=LbGYE7o8gO">
  <link rel="apple-touch-icon" sizes="114x114" href="assets/images/icons/apple-touch-icon-114x114.png?v=LbGYE7o8gO">
  <link rel="apple-touch-icon" sizes="120x120" href="assets/images/icons/apple-touch-icon-120x120.png?v=LbGYE7o8gO">
  <link rel="apple-touch-icon" sizes="144x144"

In [15]:
re.findall('Mohammad', resp.text)

[]

# Jak dostat data z takové stránky?

## Chovat se jako browser

Buď

1. pochopit, co Javascript dělá, nebo
2. opravdu Javascript spouštět

## Varianta 1. - developer tools

[![Sanctions list JSON in developer tools](sanctions2.png)](sanctions2.png)

In [16]:
resp = requests.get('https://www.sanctionsmap.eu/api/v1/regime')

In [17]:
resp.json()

{'meta': {'version': '1'},
 'response': [{'acronym': None,
   'adopted_by': {'id': 2, 'title': 'UN'},
   'amendment': 1562536800,
   'category': [],
   'country': {'code': 'AF', 'title': 'Afghanistan'},
   'court_rulings': [],
   'expiration': None,
   'general_guidances': [{'attachments': [{'name': 'ST-15530-2016-INIT-EN.pdf',
       'size': 475114,
       'type': 'application/pdf',
       'url': 'https://admin.sanctionsmap.eu/files/1oypndatvb3ci1pip5lvmtv9j/st-15530-2016-init-en.pdf'}],
     'id': 1,
     'is_direct': 0,
     'title': 'EU Best Practices for the effective implementation of restrictive measures',
     'url': 'http://data.consilium.europa.eu/doc/document/ST-8519-2018-INIT/en/pdf'},
    {'attachments': [{'name': 'ST_15598_2017_INIT_EN_Public.pdf',
       'size': 410322,
       'type': 'application/pdf',
       'url': 'https://admin.sanctionsmap.eu/files/e1dk1lv5h5c7gy2iv6vhs0s4/st-15598-2017-init-en-public.pdf'}],
     'id': 2,
     'is_direct': 0,
     'title': 'Guideli

In [24]:
regime_ids = [int(r['id']) for r in resp.json()['response'] if r['has_lists']]
regime_ids[:5]

[1, 2, 7, 9, 46]

In [25]:
regime_data = [requests.get(f'https://www.sanctionsmap.eu/api/v1/regime/{regime_id}').json() for regime_id in regime_ids]

In [31]:
idx = 0
regime_data[idx]

{'meta': {'version': '1'},
 'response': {'acronym': None,
  'adopted_by': {'id': 2, 'title': 'UN'},
  'amendment': 1562536800,
  'category': [],
  'country': {'code': 'AF', 'title': 'Afghanistan'},
  'court_rulings': [],
  'expiration': None,
  'general_guidances': [{'attachments': [{'name': 'ST-15530-2016-INIT-EN.pdf',
      'size': 475114,
      'type': 'application/pdf',
      'url': 'https://admin.sanctionsmap.eu/files/1oypndatvb3ci1pip5lvmtv9j/st-15530-2016-init-en.pdf'}],
    'id': 1,
    'is_direct': 0,
    'title': 'EU Best Practices for the effective implementation of restrictive measures',
    'url': 'http://data.consilium.europa.eu/doc/document/ST-8519-2018-INIT/en/pdf'},
   {'attachments': [{'name': 'ST_15598_2017_INIT_EN_Public.pdf',
      'size': 410322,
      'type': 'application/pdf',
      'url': 'https://admin.sanctionsmap.eu/files/e1dk1lv5h5c7gy2iv6vhs0s4/st-15598-2017-init-en-public.pdf'}],
    'id': 2,
    'is_direct': 0,
    'title': 'Guidelines on implementation 

In [33]:
current_regime = regime_data[idx]['response']
current_regime['country']

{'code': 'AF', 'title': 'Afghanistan'}

In [46]:
for measure in current_regime['measures']:
    print(measure['id'], measure['description'])
    pprint(measure['type'])
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~')

49 It is prohibited to export arms and related materiel to Zimbabwe. Related technical or financial assistance and services are also prohibited.
{'icon': 'https://www.sanctionsmap.eu/storage/icons/hYvAur5S3ZQHtBfAo6ElESAS5xl8CMUEyPrNHyMf.svg',
 'id': 4,
 'parent_id': 2,
 'title': 'Arms export'}
~~~~~~~~~~~~~~~~~~~~~~~~~~
7 All assets of the listed persons and entities should be frozen. It is also prohibited to make any funds or assets directly or indirectly available to them.
{'icon': 'https://www.sanctionsmap.eu/storage/icons/bGumIzNvSWyfmf5BJSuJGPJXR1xeGz2xrzvane19.svg',
 'id': 6,
 'title': 'Asset freeze and prohibition to make funds available'}
~~~~~~~~~~~~~~~~~~~~~~~~~~
53 Member States shall enforce travel restrictions on  persons listed in Annex I of Council Decision 2011/101/CFSP.
{'icon': 'https://www.sanctionsmap.eu/storage/icons/LUMa1Twbs3PWRgFNLhFdMcQO4ZuU9VYMHb8jrxUK.svg',
 'id': 18,
 'title': 'Restrictions on admission'}
~~~~~~~~~~~~~~~~~~~~~~~~~~
71 It is prohibited to ex

In [54]:
for regime_json in regime_data:
    current_regime = regime_json['response']
    for measure in current_regime['measures']:
        print(measure['id'], measure['type']['title'], measure['type']['id'])

28 Arms export 4
8 Asset freeze and prohibition to make funds available 6
29 Restrictions on admission 18
24 Arms export 4
2 Asset freeze and prohibition to make funds available 6
50 Restrictions on admission 18
30 Restrictions on equipment used for internal repression 19
3 Asset freeze and prohibition to make funds available 6
164 Prohibition to satisfy claims 17
34 Restrictions on admission 18
35 Arms export 4
9 Asset freeze and prohibition to make funds available 6
200 Prohibition to satisfy claims 17
36 Restrictions on admission 18
263 Asset freeze and prohibition to make funds available 6
264 Restrictions on admission 18
37 Arms embargo 2
38 Arms export 4
14 Asset freeze and prohibition to make funds available 6
165 Prohibition to satisfy claims 17
39 Restrictions on admission 18
1 Asset freeze and prohibition to make funds available 6
4 Asset freeze and prohibition to make funds available 6
46 Restrictions on admission 18
15 Asset freeze and prohibition to make funds available 6


In [63]:
aggregate_sanction_list = []

for regime_json in regime_data:
    current_regime = regime_json['response']
    for measure in current_regime['measures']:
        if measure['type']['id'] == 6:  # Funds freeze
            for sanction_list in measure['lists']:
                aggregate_sanction_list.extend(sanction_list['members'])

aggregate_sanction_list = pd.DataFrame(aggregate_sanction_list)
aggregate_sanction_list

,FSD_ID,creation_date,id_code,name,order_by,programme,reason,suspend,suspension_end_date,type
0,505.0,9.803772e+08,None,"Abdul Hai Hazem Abdul Qader, Abdul Hai Hazem",0.0,AFG,None,NaN,NaN,P
1,513.0,9.803772e+08,None,"Agha, Abdul Rahman",0.0,AFG,None,NaN,NaN,P
2,515.0,9.828828e+08,None,"Sayed Mohammad Azim Agha, Sayed Mohammad Azim ...",0.0,AFG,None,NaN,NaN,P
3,516.0,9.808956e+08,None,"Sayyed Ghiassouddine Agha, Sayed Ghias, Sayed ...",0.0,AFG,None,NaN,NaN,P
4,517.0,9.828828e+08,None,Mohammad Ahmadi,0.0,AFG,None,NaN,NaN,P
5,521.0,9.803772e+08,None,"Haji Ahmad Jan, Ahmed Jan Akhund, Ahmed Jan Ak...",0.0,AFG,None,NaN,NaN,P
6,522.0,9.803772e+08,None,Mohammad Essa Akhund,0.0,AFG,None,NaN,NaN,P
7,523.0,9.828828e+08,None,Attiqullah Akhund,0.0,AFG,None,NaN,NaN,P
8,524.0,9.808956e+08,None,"Akhund, Shahidwror, Allahdad, Allah Dad Matin",0.0,AFG,None,NaN,NaN,P
9,525.0,9.803772e+08,None,"Ubaidullah Akhund Yar Mohammed Akhund, Obaidul...",0.0,AFG,None,NaN,NaN,P


## Varianta 2. - opravdový browser

[Puppeteer](https://pptr.dev/) - ovládání Chrome přes protokol Developer Tools

- Prakticky vše, co může dělat člověk v browseru
- Screenshoty, naviagace, scrollování, úprava HTML, odesílání formulářů, změna velikosti okna...
- V Javascriptu

[Pyppeteer](https://github.com/miyakogi/pyppeteer) - port Puppeteera do Pythonu

- `python3 -m pip install pyppeteer && pyppeteer-install` (druhý příkaz stáhne Chromium ~100 MB)
- Programuje se pomocí async API (viz. další slide)

In [89]:
import asyncio
from pyppeteer import launch

elements = []

async def get_single_sanction_list():
    browser = await launch()
    page = await browser.newPage()
    await page.goto('https://sanctionsmap.eu/#/main/details/1/lists?search=%7B%22value%22:%22%22,%22searchType%22:%7B%7D%7D1')
    await page.screenshot({'path': 'sanctions-afghanistan1.png'})
    await page.waitForSelector('li[data-heading="Name"] div')
    await page.screenshot({'path': 'sanctions-afghanistan2.png'})
    
    for el in await page.querySelectorAll('li[data-heading="Name"] div'):
        elements.append(await page.evaluate('(el) => el.textContent', el))

    await browser.close()

asyncio.get_event_loop().run_until_complete(get_single_sanction_list())

In [90]:
elements

['\n  Abdul Hai Hazem Abdul Qader, Abdul Hai Hazem\n',
 '\n  Agha, Abdul Rahman\n',
 '\n  Sayed Mohammad Azim Agha, Sayed Mohammad Azim Agha, Agha Saheb\n',
 '\n  Sayyed Ghiassouddine Agha, Sayed Ghias, Sayed Ghiasuddin Sayed Ghousuddin, Sayyed Ghayasudin\n',
 '\n  Mohammad Ahmadi\n',
 '\n  Haji Ahmad Jan, Ahmed Jan Akhund, Ahmed Jan Akhundzada Wazir\n',
 '\n  Mohammad Essa Akhund\n',
 '\n  Attiqullah Akhund\n',
 '\n  Akhund, Shahidwror, Allahdad, Allah Dad Matin\n',
 '\n  Ubaidullah Akhund Yar Mohammed Akhund, Obaidullah Akhund, Obaid Ullah Akhund\n',
 '\n  Mohammad Abbas Akhund\n',
 '\n  Aminullah Amin Quddus, Muhammad Yusuf, Aminullah Amin\n',
 '\n  Nazirullah Hanafi Waliullah, Nazirullah Aanafi Waliullah\n',
 '\n  Muhammad Taher Anwari, Mohammad Taher Anwari, Mohammad Tahre Anwari, Haji Mudir, Muhammad Tahir Anwari\n',
 '\n  Arefullah Aref Ghazi Mohammad, Arefullah Aref\n',
 '\n  Sayed Esmatullah Asem Abdul Quddus, Asmatullah Asem, Sayed Esmatullah Asem, Esmatullah Asem\n',
 '\n  A